# Scrap them all !

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request, response
import re

path_file = '/media/flo/Stockage/Data/ScrapMyMinistres/'

In [2]:
%qtconsole

/usr/local/lib/python3.4/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [3]:
def clean_ministres_fonctions(fonction_ministres):
    fonction_ministres_clean =  [x for x in fonction_ministres if any([len(x) < 5, '[' not in x ])]
    fonction_ministres_clean = '#'.join(fonction_ministres_clean).split('##')
    fonction_ministres_clean = [x.replace('#', ' / ') for x in fonction_ministres_clean]

    return fonction_ministres_clean

def clean_ministres_wiki(wiki_ministre) : 
    wiki_ministres_clean = [x for x in wiki_ministre if ('France' not in x) & 
                            ('jpg' not in x) & ('Fichier' not in x) & any(['(' in x or 'Conway' in x, len(x) <= 35]) &  # Philipe martin (homme politique, ...)
                            ('gauche' not in x) & ('Outre-' not in x) & ('Ville' not in (x)) &
                            ('Logement' not in x) & ('Budget' not in x)  & ('Walwari' not in x) &
                            ('#cite' not in x) & ('tiquette' not in x) & ('cologistes' not in x) & ('cologie' not in x)]
    print('%s ministres trouvés' % (len(wiki_ministres_clean)))
    
    return wiki_ministres_clean
# le any(['(' in x or 'Conway' in x, len(x) <= 36]) sert à conserver certains wiki atypiques (Conway = très long, Philippe Martin = (homme polituque..))

def get_my_gouv(link_gouv):
    # Request
    header = {'User-Agent': 'Mozilla/5.0'} # Needed to prevent 403 error on Wikipedia
    page = request.Request(link_gouv, headers=header)
    html = request.urlopen(page) 
    soup = BeautifulSoup(html, "lxml")
    cells = soup.findAll("table", {"class": "wikitable sortable"})
    #  Init
    wiki_ministres = []
    fonction_ministres = [] 
    n_ministres = []
    n_min = 0
    # Scraping
    for cell in cells: # Tableau principal
        cells_href = cell.findAll("a", href=True)
        for c_href in cells_href: # éléments de chaque tableau
            wiki_ministres.append(c_href['href'])
            fonction_ministres.append(c_href.get_text())
            n_ministres.append(n_min)
            n_min += 1
    wiki_ministres_clean = clean_ministres_wiki(wiki_ministres)
    fonction_ministres_clean = clean_ministres_fonctions(fonction_ministres)
    return (fonction_ministres_clean, wiki_ministres_clean)

In [4]:
dict_holland = {'holland_ayrault_1': "/wiki/Gouvernement_Jean-Marc_Ayrault_(1)",
                'holland_ayrault_2': "/wiki/Gouvernement_Jean-Marc_Ayrault_(2)",
                'holland_valls_1': "/wiki/Gouvernement_Manuel_Valls_(1)",
                'holland_valls_2': "/wiki/Gouvernement_Manuel_Valls_(2)" }

all_gouv = pd.DataFrame(columns=['gouvernement', 'wiki', 'fonctions'])
for keys, values in dict_holland.items():
    gouv = pd.DataFrame()
    (fonction_ministres_clean, wiki_ministres_clean) = get_my_gouv('https://fr.wikipedia.org' + values)
    gouv["wiki"] = wiki_ministres_clean
    gouv["fonctions"] = fonction_ministres_clean
    gouv["gouvernement"] = keys
    all_gouv = pd.concat([all_gouv, gouv])
    
# -- Export
all_gouv = all_gouv.ix[:, ['gouvernement', 'wiki', 'fonctions']] # changement de l'ordre des colonnes
all_gouv.to_csv(path_file + 'gouvernements_ministres.csv', sep=';',encoding='utf-8')

39 ministres trouvés
35 ministres trouvés
32 ministres trouvés
38 ministres trouvés


In [201]:
all_gouv.head(3)

,gouvernement,wiki,fonctions
0,holland_valls_2,/wiki/Manuel_Valls,/ Premier ministre / Manuel Valls / PS
1,holland_valls_2,/wiki/Jean-Marc_Ayrault,Ministre des Affaires étrangères et du Développement international / Jean-Marc Ayrault / PS
2,holland_valls_2,/wiki/S%C3%A9gol%C3%A8ne_Royal,"Ministre de l'Environnement, de l'Énergie et de la Mer, chargée des relations internationales sur le climat / Ségolène Royal / PS"


## Test
Vérifions qu'il ya bien autant de ministres que de fonction. Visuellement.

In [5]:
(fonction_ministres_clean, wiki_ministres_clean) = get_my_gouv("https://fr.wikipedia.org" + dict_holland["holland_valls_2"])

39 ministres trouvés


In [6]:
pd.options.display.max_colwidth = 300
df = pd.DataFrame()
df['wiki_ministres'] = wiki_ministres_clean
df1 = pd.DataFrame()
df1['fonction_ministres'] = pd.Series(fonction_ministres_clean)

df.merge(df1, left_index=True, right_index=True, how='outer')  # changer 'left' ou 'right' pour identifier un éventuel pb
#df['fonctions_ministres'] = fonction_ministres_clean

,wiki_ministres,fonction_ministres
0,/wiki/Manuel_Valls,/ Premier ministre / Manuel Valls / PS
1,/wiki/Jean-Marc_Ayrault,Ministre des Affaires étrangères et du Développement international / Jean-Marc Ayrault / PS
2,/wiki/S%C3%A9gol%C3%A8ne_Royal,"Ministre de l'Environnement, de l'Énergie et de la Mer, chargée des relations internationales sur le climat / Ségolène Royal / PS"
3,/wiki/Najat_Vallaud-Belkacem,Ministre de l'Éducation nationale / de l'Enseignement supérieur et de la Recherche / Najat Vallaud-Belkacem / PS
4,/wiki/Michel_Sapin,Ministre des Finances et des Comptes publics / Michel Sapin / PS
5,/wiki/Marisol_Touraine,Ministre des Affaires sociales et de la Santé / Marisol Touraine / PS
6,/wiki/Jean-Yves_Le_Drian,Ministre de la Défense / Jean-Yves Le Drian / PS
7,/wiki/Jean-Jacques_Urvoas,"Garde des Sceaux, ministre de la Justice / Jean-Jacques Urvoas / PS"
8,/wiki/Myriam_El_Khomri,"Ministre du Travail, de l'Emploi, de la Formation professionnelle et du Dialogue social / Myriam El Khomri / PS"
9,/wiki/Jean-Michel_Baylet,"Ministre de l'Aménagement du territoire, de la Ruralité et des Collectivités territoriales / Jean-Michel Baylet / PRG"


###  * Nettoyage sur LibreOffice * 

Clean des colonnes su LibreOffice car texte parfois trop non-struturé.

In [7]:
all_gouv_clean = pd.read_csv(path_file + 'gouvernements_ministres_clean.csv', sep=';', header=0)
all_gouv_clean.columns = ['gouvernement', 'wiki', 'fonctions', 'Nom', 'Parti']

In [8]:
all_gouv_clean.head(3)

,gouvernement,wiki,fonctions,Nom,Parti
0,holland_valls_2,/wiki/Manuel_Valls,Premier ministre,Manuel Valls,PS
1,holland_valls_2,/wiki/Jean-Marc_Ayrault,Ministre des Affaires étrangères et du Développement international,Jean-Marc Ayrault,PS
2,holland_valls_2,/wiki/S%C3%A9gol%C3%A8ne_Royal,"Ministre de l'Environnement, de l'Énergie et de la Mer, chargée des relations internationales sur le climat",Ségolène Royal,PS


## Looping the ministres

## select a ministre


In [9]:
dict_holland_char = {'holland_ayrault_1': "/wiki/Gouvernement_Jean-Marc_Ayrault_\(1\)",
                    'holland_ayrault_2': "/wiki/Gouvernement_Jean-Marc_Ayrault_\(2\)",
                    'holland_valls_1': "/wiki/Gouvernement_Manuel_Valls_\(1\)",
                    'holland_valls_2': "/wiki/Gouvernement_Manuel_Valls_\(2\)" }

In [10]:
def get_lifespan_ministres(gouv_name):
    (fonction_ministres_clean, wiki_ministres_clean) = get_my_gouv('https://fr.wikipedia.org' + dict_holland[gouv_name])
    all_dates = []
    all_birthdays = []
    all_names = []
    for ministre in wiki_ministres_clean:
        page_ministre = "https://fr.wikipedia.org" + ministre
        header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
        page = request.Request(page_ministre, headers=header)
        html = request.urlopen(page)
        soup = BeautifulSoup(html, "lxml")
        cells = soup.find('table', {'class': 'infobox_v2'})  # selection du tableau de droite
        # -- Names
        all_names.append(cells.find('td', {'class': 'entete defaut'}).getText())
        # -- Birthdays
        all_birthdays.append(cells.find('time', {'class': 'nowrap date-lien bday'})['datetime'])
        # -- Lifespan
        cells_gouv = cells.find('a', href=re.compile(dict_holland_char[gouv_name]))  # selection du gouvernement
        l_dates = []
        try: # recherche de(s) (la) date(s) immédiatement au dessus de 'gouvernement'
            cells_date = cells_gouv.findPrevious('b').findAll('time')  
            for c_dates in cells_date:
                l_dates.append(c_dates['datetime'])
        except AttributeError:
            print("pas de date trouvée pour le ministre %s" % (ministre))
            l_dates.append("Non_trouvee")
        all_dates.append(l_dates)

    # -- clean date
    date_debut = [x[0] for x in all_dates]
    date_fin = [x[1] if (len(x) == 2) else pd.datetime.today().date() for x in all_dates]
    # -- Results table    
    df_date = pd.DataFrame()
    df_date['wiki'] = wiki_ministres_clean
    df_date['Nom'] = all_names
    df_date['birthday'] = pd.to_datetime(all_birthdays, errors='coerce')
    df_date['date_debut'] = pd.to_datetime(date_debut, errors='coerce')
    df_date['date_fin'] = pd.to_datetime(date_fin, errors='coerce')
    df_date['duree_mendat'] = df_date['date_fin'] - df_date['date_debut']
    df_date['gouvernement'] = gouv_name
    df_date = df_date.ix[:, ['gouvernement', 'wiki', 'Nom', 'birthday', 'date_debut', 'date_fin', 'duree_mendat']]
    return df_date

## Run

In [11]:
all_lifespan = pd.DataFrame()
for keys, values in dict_holland_char.items():
    df_date = get_lifespan_ministres(keys)
    all_lifespan = pd.concat([df_date, all_lifespan])
    all_lifespan.to_csv(path_file + 'ministres_lifespan.csv', sep=';')

39 ministres trouvés
pas de date trouvée pour le ministre /wiki/Juliette_M%C3%A9adel
35 ministres trouvés
32 ministres trouvés
38 ministres trouvés


In [12]:
all_lifespan.head()

,gouvernement,wiki,Nom,birthday,date_debut,date_fin,duree_mendat
0,holland_ayrault_2,/wiki/Jean-Marc_Ayrault,Jean-Marc Ayrault,1950-01-25,2012-05-15,2014-03-31,685 days
1,holland_ayrault_2,/wiki/Laurent_Fabius,Laurent Fabius,1946-08-20,2012-05-16,2016-02-11,1366 days
2,holland_ayrault_2,/wiki/Vincent_Peillon,Vincent Peillon,1960-07-07,2012-05-16,2014-03-31,684 days
3,holland_ayrault_2,/wiki/Christiane_Taubira,Christiane Taubira,1952-02-02,2012-05-16,2016-01-27,1351 days
4,holland_ayrault_2,/wiki/Pierre_Moscovici,Pierre Moscovici,1957-09-16,2012-05-16,2014-03-31,684 days


# Merge

In [236]:
all_gouv_clean.head(1)

,gouvernement,wiki,fonctions,Nom,Parti
0,holland_valls_2,/wiki/Manuel_Valls,Premier ministre,Manuel Valls,PS


In [237]:
all_lifespan.head(1)

,gouvernement,wiki,Nom,birthday,date_debut,date_fin,duree_mendat
0,holland_ayrault_2,/wiki/Jean-Marc_Ayrault,Jean-Marc Ayrault,1950-01-25,2012-05-15,2014-03-31,685 days


In [13]:
table_finale = pd.merge(all_lifespan,
                        all_gouv_clean[['gouvernement', 'wiki', 'fonctions', 'Parti']],
                        on=['gouvernement', 'wiki'])
# -- Reorder columns
table_finale = table_finale.ix[: ,['gouvernement', 'wiki', 'Nom', 'fonctions',
                                   'birthday', 'date_debut', 'date_fin',
                                   'duree_mendat', 'Parti']]

In [244]:
table_finale.head(2)

,gouvernement,wiki,Nom,fonctions,birthday,date_debut,date_fin,duree_mendat,Parti
0,holland_ayrault_2,/wiki/Jean-Marc_Ayrault,Jean-Marc Ayrault,Premier ministre,1950-01-25,2012-05-15,2014-03-31,685 days,PS
1,holland_ayrault_2,/wiki/Laurent_Fabius,Laurent Fabius,Ministre des Affaires étrangères,1946-08-20,2012-05-16,2016-02-11,1366 days,PS


## Export

In [245]:
table_finale.to_csv(path_file + 'table_finale.csv', sep=';', encoding='utf-8')

# Sandbox

## Test fonction

In [190]:
df_date_holland_ayrault_1 = get_lifespan_ministres('holland_ayrault_1')

35 ministres trouvés


In [193]:
df_date_holland_ayrault_1.head(5)

,wiki,Nom,birthday,date_debut,date_fin,duree_mendat,gouvernement
0,/wiki/Jean-Marc_Ayrault,Jean-Marc Ayrault,1950-01-25,2012-05-15,2014-03-31,685 days,holland_ayrault_1
1,/wiki/Laurent_Fabius,Laurent Fabius,1946-08-20,2012-05-16,2016-02-11,1366 days,holland_ayrault_1
2,/wiki/Vincent_Peillon,Vincent Peillon,1960-07-07,2012-05-16,2014-03-31,684 days,holland_ayrault_1
3,/wiki/Christiane_Taubira,Christiane Taubira,1952-02-02,2012-05-16,2016-01-27,1351 days,holland_ayrault_1
4,/wiki/Pierre_Moscovici,Pierre Moscovici,1957-09-16,2012-05-16,2014-03-31,684 days,holland_ayrault_1


## Test scrap

In [179]:
for ministre in wiki_ministres_clean[:1]:
    page_ministre = "https://fr.wikipedia.org" + ministre
    header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
    page = request.Request(page_ministre, headers=header)
    html = request.urlopen(page)
    soup = BeautifulSoup(html, "lxml")
    cells = soup.find('table', {'class': 'infobox_v2'})  # selection du tableau de droite
    cells_gouv = cells.find('a', href=re.compile(dict_holland_char['holland_ayrault_1']))  # selection du gouvernement
    l_dates = []


In [145]:
# -- date_naissance
cells.find('time', {'class': 'nowrap date-lien bday'})['datetime']

'1950-01-25'

In [183]:
# -- Nom
cells.find('td', {'class': 'entete defaut'}).getText()

'Jean-Marc Ayrault'